<a href="https://colab.research.google.com/github/cdelaoutre/rerA-waze/blob/master/pythonTests/scrapTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports ###

In [1]:
!pip install tweepy
import tweepy
import json
import pprint
import datetime as dt
import pandas as pd

*Twitter API Connection Infos manually loaded*

In [2]:
def connectTweeterApi(twitterInfos):
    # Load Twitter API secrets from an external JSON file
    with open(twitterInfos, "r") as read_file:
        connInfos = json.load(read_file)

    api_key = connInfos['twitter_key']
    api_secret_key = connInfos['twitter_secret']
    access_token = connInfos['twitter_accessToken']
    access_token_secret = connInfos['twitter_accessTokenSecret']
    # Connect to Twitter API using the secrets
    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_token_secret)
    twitterAPI = tweepy.API(auth)
    return twitterAPI
twitterAPI = connectTweeterApi("twitterInfos.json")

In [3]:
target = 'RER_A'
scrappedAccount = twitterAPI.get_user(target)
#pprint.pprint(user._json)

In [4]:
print("name: " + scrappedAccount.name)
print("screen_name: " + scrappedAccount.screen_name)
print("description: " + scrappedAccount.description)
print("statuses_count: " + str(scrappedAccount.statuses_count))
print("friends_count: " + str(scrappedAccount.friends_count))
print("followers_count: " + str(scrappedAccount.followers_count))

name: RER A
screen_name: RER_A
description: Bienvenue sur le compte du #RERA ! Suivez le trafic en temps réel, la vie et les coulisses de votre ligne. Avec vous 5j/7 de 6h à 20h.
statuses_count: 78996
friends_count: 92
followers_count: 158016


In [5]:
tweets = scrappedAccount.statuses_count
account_created_date = scrappedAccount.created_at
delta = dt.datetime.utcnow() - account_created_date
account_age_days = delta.days
print("Account age (in days): " + str(account_age_days))
if account_age_days > 0:
    print("Average tweets per day: " + "%.2f"%(float(tweets)/float(account_age_days)))

Account age (in days): 2938
Average tweets per day: 26.89


In [14]:
def scrapTweet(tweet, end_date, dest):
    #Stop when max date is reached
    if tweet.created_at < end_date:
        return "Tweet out of date range"
    #Check if its an answer to exclude it
    if hasattr(tweet, 'entities'):
        if 'user_mentions' in tweet.entities.keys():
            if len(tweet.entities['user_mentions']) > 0:
                return "Answer identified"
    #Grab the Tweet
    if hasattr(tweet, 'text'):
        dest['Tweet'].append(tweet.text)
        #Grab the date (if parsable, else None)
        if hasattr(tweet, 'created_at'):
            dest['Date'].append(tweet.created_at)

def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [15]:
scrappedTweets ={
    'Tweet' : [],
    'Date' : []
}
end_date = dt.datetime.utcnow() - dt.timedelta(days=365)
for tweet in limit_handled(tweepy.Cursor(twitterAPI.user_timeline, id=target).items()):
    scrapTweet(tweet, end_date, scrappedTweets)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'limit_handled' raised StopIteration
  


In [16]:
myTweets = pd.DataFrame(scrappedTweets)
myTweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Tweet   782 non-null    object        
 1   Date    782 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 12.3+ KB


In [10]:
pd.DataFrame(scrappedTweets).to_csv('tweets.csv')